In [1]:
import logging
import time
from typing import Any, Callable, TypeVar, Union
from neopipe.result import Err, Ok, Result
from neopipe.task import Task, FunctionTask, ContainerTask, ContainerTaskBase
from neopipe.pipeline import Pipeline

In [2]:

logger = logging.getLogger(__name__)

T = TypeVar("T")
E = TypeVar("E")


In [3]:
# Define a simple function task
def simple_task(x: int) -> Result[int, str]:
    return Ok(x + 1)


# Define a container task class
class MultiplyTask(ContainerTaskBase):
    def __init__(self, factor: int):
        self.factor = factor

    def __call__(self, x: int) -> Result[int, str]:
        return Ok(x * self.factor)


# Create a pipeline
pipeline = Pipeline()

# Register tasks using the decorator
@pipeline.register(retries=2)
def add_task(x: int) -> Result[int, str]:
    return Ok(x + 2)


# Register a container task
multiply_task_instance = MultiplyTask(3)
pipeline.append_task(ContainerTask(multiply_task_instance))

# Run the pipeline
result = pipeline.run(1, show_progress=True)
print(result.unwrap())  # Should print 9 ((1 + 2) * 3)


2024-07-14 03:26:09 - neopipe.pipeline - INFO - Pipeline started (UUID: 82d1f203-699e-4a4f-8ca8-9df87ddfc7ca)
2024-07-14 03:26:09 - neopipe.task - INFO - Function add_task succeeded on attempt 1
2024-07-14 03:26:09 - neopipe.pipeline - INFO - 1/2 is complete
2024-07-14 03:26:09 - neopipe.task - INFO - Container MultiplyTask succeeded on attempt 1
2024-07-14 03:26:09 - neopipe.pipeline - INFO - 2/2 is complete


Callable <function add_task at 0x111018540>
9


In [4]:
# Define a container task class
class MultiplyTask(ContainerTaskBase):
    def __init__(self, factor: int):
        self.factor = factor

    def __call__(self, x: int) -> Result[int, str]:
        return Ok(x * self.factor)

# Create a pipeline
pipeline = Pipeline()

# Register the container task using the decorator
@pipeline.register(retries=2)
class MultiplyTask(ContainerTaskBase):
    def __init__(self, factor: int = 3):
        self.factor = factor

    def __call__(self, x: int) -> Result[int, str]:
        return Ok(x * self.factor)

# Run the pipeline
result = pipeline.run(2, show_progress=True)
print(result.unwrap())  # Should print 6 if factor is set to 3 in MultiplyTask's init


2024-07-14 03:26:09 - neopipe.pipeline - INFO - Pipeline started (UUID: c53baeb6-4d6d-4449-984a-3d0a109d0f51)
2024-07-14 03:26:09 - neopipe.task - INFO - Container MultiplyTask succeeded on attempt 1
2024-07-14 03:26:09 - neopipe.pipeline - INFO - 1/1 is complete


Class <class '__main__.MultiplyTask'>
6


In [5]:
pipeline = Pipeline()

@pipeline.register(retries=1)
class InvalidTask:
    pass

Callable <class '__main__.InvalidTask'>


In [6]:
class AnotherInvalidTask:
    pass

callable(AnotherInvalidTask)

True